In [1]:
from seagul.rl.run import run_sg
from seagul.rl.models import SwitchedPPOModel
from seagul.nn import MLP
from seagul.rl.ppo_switching import ppo_switch

import torch
import torch.nn as nn

import gym

## init policy, valuefn
input_size = 4
output_size = 1
layer_size = 64
num_layers=3
activation=nn.ReLU

torch.set_default_dtype(torch.double)

from seagul.sims.cartpole import LQRControl
env_name = 'su_cartpole-v0'

model = SwitchedPPOModel(
    policy = MLP(input_size, output_size, num_layers, layer_size, activation),
    value_fn = MLP(input_size, 1, num_layers, layer_size, activation),
    gate_fn = MLP(input_size, 1, num_layers,layer_size, activation),
    nominal_policy=LQRControl,
    action_var = 4,
    gate_var = .08,
    env = gym.make(env_name)
)

arg_dict = {
    'env_name' : env_name,
    'model' : model,
    'num_epochs' : 5,
    'action_var_schedule' : [.1],
}

run_sg(arg_dict, ppo_switch)


Logging to /var/folders/qq/gpxz4l6s1tndfdhysbz8bdym0000gn/T/openai-2019-09-03-20-29-45-861767


please enter a name for this run:  git_sha5
please enter a brief description of the run:  


100%|██████████| 5/5 [00:55<00:00, 11.04s/it]


In [23]:
import subprocess
label = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()

In [24]:
label

'5c9e66c428a42c6f23e50f15c96fb4106771c93c'

In [1]:
import gym
import seagul.envs

env_name = 'su_acro_drake-v0'
env = gym.make(env_name)

from seagul.rl.run_utils import run_sg
from seagul.rl.models import SwitchedPPOModel
from seagul.nn import MLP
from seagul.rl.algos.ppo2_switching import ppo_switch

import torch
import torch.nn as nn

import gym

## init policy, valuefn
input_size = 4
output_size = 1
layer_size = 64
num_layers=2
activation=nn.ReLU

torch.set_default_dtype(torch.double)

from seagul.sims.cartpole import LQRControl
env_name = 'su_drake_acro-v0'

def control(q):
    k = np.array([[1316.85000612,  555.41763935,  570.32667002,  272.57631536]])
    #k = np.array([[278.44223126, 112.29125985, 119.72457377,  56.82824017]])
    gs = np.array([pi,0,0,0])
    #return 0
    return (-k.dot(gs - np.asarray(q))).squeeze()


model = SwitchedPPOModelActHold(
    #policy = MLP(input_size, output_size, num_layers, layer_size, activation),
    policy = torch.load("policy_warm_final"),
    value_fn = torch.load("value_warm_final"),
    #MLP(input_size, 1, num_layers, layer_size, activation),
    gate_fn  = torch.load("gate_fn_dr"),
    nominal_policy=control,
    hold_count = 200,
)

    
arg_dict = {
    'env_name' : env_name,
    'model' : model,
    'num_epochs' : 500,
    'epoch_batch_size': 2048,
    'action_var_schedule' : [2,2],
    'gate_var_schedule'   : [.1,.1],
    'gamma' : 1,
    'seed': 0,
}


run_sg(arg_dict, ppo_switch, 'testtest', 'test')



ImportError: No module named seagul.envs